<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Deep-Learning-Model-Visualization/blob/main/Chapter03/DLMV_Chapter03_08_Optimizers_Plotly.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Chapter 03 - 08 - Deep-Learning Optimizers with Plotly

## Version

In [1]:
import plotly
plotly.__version__

'5.15.0'

## Data

In [2]:
import numpy as np
np.random.seed(2)

def f(x):
  return 2*x - 3 + (np.random.rand(len(x))-0.5)*2
x = np.random.randint(-20, 20, 20)
y = f(x)

# Visualizations

In [3]:
import plotly.graph_objects as go
# Enable this if running on Google Colab
from google.colab import output
output.enable_custom_widget_manager()

In [4]:
def plot_losses(losses):
  # Step 1: create the loss trace
  line = go.Scatter(y=losses, mode='lines')
  layout = dict(
      title=dict(text='Losses',x=0.5,y=0.9),
      xaxis=dict(title="iteration"),
      yaxis=dict(title="loss", range=[-1, 20]),
      width=400, height=400,
      margin=dict(l=50, r=20, t=70, b=50)
  )
  fig = go.Figure(data=[line], layout=layout)
  # Step 2: add the annotation
  msg = f'Final loss: {round(losses[-1], 3)}'
  fig.add_annotation(
      x=len(losses)-1, y=losses[-1],
      text=msg,
      showarrow=True, arrowhead=1,
      xanchor='right')

  return fig

In [5]:
def generate_3d_data(X, Y, forward_fn, loss_fn):
  grid_size = 100
  theta1s = np.linspace(0.0, 3, grid_size)
  theta2s = np.linspace(1, -4, grid_size)
  Z = np.zeros((grid_size, grid_size))
  for row in range(grid_size):
    for col in range(grid_size):
      theta = np.array([[theta1s[col]], [theta2s[row]]])
      Y_hat = forward_fn(theta, X)
      loss = loss_fn(Y_hat, Y)
      Z[row,col] = loss
  return theta1s, theta2s, Z

In [6]:
def plot_gradients(thetas, losses):
  # Step 1: create the surface trace
  x, y, z = generate_3d_data(X, Y, forward, mse)
  surface = go.Surface(x = x, y = y, z = z,
                       showscale=False)
  # Step 2: create the gradient trace
  gradients = go.Scatter3d(
      x=thetas[:,0,:].flatten(),
      y=thetas[:,1,:].flatten(),
      z=losses,
      mode='lines',
      line={'width': 10})
  # Step 3: configure the layout and create the figure
  layout = dict(
      title=dict(text='Gradients',x=0.5,y=0.9),
      scene = dict(
          xaxis=dict(title='theta 1'),
          yaxis=dict(title='theta 2'),
          zaxis=dict(title="loss")),
      width=400, height=400,
      margin=dict(l=20, r=20, t=20, b=20)
  )
  fig = go.Figure([surface, gradients], layout=layout)

  return fig

## Deep learning from scratch

In [7]:
# reshape Y
Y = np.reshape(y, (-1,1))
# add bias
X = np.array(list(zip(x, np.ones_like(x))))

In [8]:
# forward
def forward(theta, X):
  Y_hat = np.matmul(X, theta)
  return Y_hat
# loss
def mse(Y_hat, Y):
  return np.mean((Y_hat - Y)**2)
# derrivative
def gradient(X, Y, Y_hat):
  return (2*np.matmul((Y_hat-Y).T, X)/len(Y)).T
# compute loss for theta
def compute_loss(theta, X, Y):
  Y_hat = forward(theta, X)
  loss = mse(Y_hat, Y)
  return loss

## Gradient Descent

In [9]:
class GD():
  def __init__(self, lr):
    self.lr = lr
  def backward(self, theta, gradients):
    theta = theta - self.lr*(gradients)
    return theta

In [10]:
def train_step(X, Y, theta, optimizer):
  # forward
  Y_hat = forward(theta, X)
  # backward
  dLdTheta = gradient(X, Y, Y_hat)
  theta = optimizer.backward(theta, dLdTheta)
  return theta, dLdTheta

In [11]:
def train(X, Y, theta0, optimizer, num_epochs):
  losses, thetas, dLdThetas = [], [], []
  theta = theta0
  for epoch in range(num_epochs):
    theta, dLdTheta=train_step(X, Y, theta, optimizer)
    loss = compute_loss(theta, X, Y)
    thetas.append(theta)
    losses.append(loss)
    dLdThetas.append(dLdTheta)
  return np.array(losses), np.array(thetas), np.array(dLdThetas)

# Training

In [12]:
theta0 = np.random.rand(2, 1)
num_epochs = 100
lr = 0.001
optimizer = GD(lr = lr)
losses, thetas, dLdThetas = train(X, Y, theta0,
                                  optimizer=optimizer,
                                  num_epochs=num_epochs)


In [13]:
import ipywidgets as widgets
figLosses = go.FigureWidget(plot_losses(losses))
figGrads = go.FigureWidget(plot_gradients(thetas, losses))
widgets.HBox([figLosses, figGrads])

    'data': [{'mode': 'lines',
              'type': 'scatter',
              'u…

## Stochastic Gradient Descent

In [14]:
# train per mini batch
def train_mini_batch(X, Y, theta0, optimizer, num_epochs, batch_size):
  batches = len(X)//batch_size
  indices = np.arange(len(X))
  losses, thetas, dLdThetas = [], [], []
  theta = theta0
  for e in range(num_epochs):
    # shuffle
    np.random.shuffle(indices)
    for i in range(batches):
      start = i*batch_size
      stop = start + batch_size
      indices_b = indices[start:stop]
      X_b = X[indices_b]
      Y_b = Y[indices_b]
      theta, dLdTheta = train_step(X_b, Y_b, theta, optimizer)
      thetas.append(theta)
      loss = compute_loss(theta, X, Y)
      losses.append(loss)
      dLdThetas.append(dLdTheta)
  return np.array(losses), np.array(thetas), np.array(dLdThetas)


In [15]:
optimizer = GD(lr = lr)
losses, thetas, dLdThetas = train_mini_batch(X, Y, theta0,
                                  optimizer=optimizer,
                                  num_epochs=num_epochs,
                                  batch_size=5)

In [16]:
import ipywidgets as widgets
figLosses = go.FigureWidget(plot_losses(losses))
figGrads = go.FigureWidget(plot_gradients(thetas, losses))
widgets.HBox([figLosses, figGrads])

    'data': [{'mode': 'lines',
              'type': 'scatter',
              'u…

In [17]:
class Momentum():
  def __init__(self, lr, moment, nesterov=False):
    self.lr = lr
    self.moment = moment
    self.first_step = True
    self.nesterov = nesterov

  def backward(self, theta, gradients):
    if self.first_step:
      self.v = np.zeros_like(theta)
      self.first_step = False

    if not self.nesterov:
      self.v = self.moment*self.v + self.lr * gradients
      theta = theta - self.v
    else:
      self.v = self.moment*self.v + gradients
      theta = theta - self.lr * self.v

    return theta

In [18]:
optimizer = Momentum(lr = lr, moment=0.9)
losses, thetas, dLdThetas = train_mini_batch(X, Y, theta0, optimizer=optimizer, num_epochs=num_epochs, batch_size = 5)

In [19]:
import ipywidgets as widgets
figLosses = go.FigureWidget(plot_losses(losses))
figGrads = go.FigureWidget(plot_gradients(thetas, losses))
widgets.HBox([figLosses, figGrads])

    'data': [{'mode': 'lines',
              'type': 'scatter',
              'u…

# Deployment

In [20]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 10.9 MB/s eta 0:00:00


In [21]:
# Step 1: Import and initialize the app
# Import packages
from dash import Dash, html, dcc, callback, Input, Output

# Initialize the app
app = Dash(__name__)
# Step 2: Create a header and a dropdown list
# Create a header
header = html.H1(children='Deep Learning Optimizers')
# Create a dropdown
dropdown = html.Label(
    ["Select one: ",
    dcc.Dropdown(
      id= 'optimizer',
      options = [
        {"label": "Gradient Descent", "value": "GD"},
        {"label": "Stochastic GD", "value": "SGD"},
        {"label": "Momentum", "value": "Momentum"}
        ]
    )])
# Step 3: Create two columns for the two figures
figLossesDv = html.Div(
    dcc.Graph(figure={}, id='figLosses'),
    style=dict(flex="50%"))
figGradsDv = html.Div(
    dcc.Graph(figure={}, id='figGrads'),
    style=dict(flex="50%"))
figuresDv = html.Div(
    children=[figLossesDv, figGradsDv],
    style=dict(display="flex"))

# Step 4: Create the app layout
app.layout = html.Div([header, dropdown, figuresDv])

# Step 5: Create the callback
output1 = Output(component_id='figLosses',
                 component_property='figure')
output2 = Output(component_id='figGrads',
                 component_property='figure')
input = Input(component_id='optimizer',
              component_property='value')
@callback([output1, output2],[input])
def update_figure(optimizer):
  # Step 1: Train the deep learning model
  theta0 = np.random.rand(2, 1)
  num_epochs = 100
  lr = 0.001
  if optimizer == "GD":
    # code for training model using GD
    optimizer = GD(lr = lr)
    losses, thetas, _ = train(X, Y, theta0,
                                      optimizer=optimizer,
                                      num_epochs=num_epochs)
  elif optimizer == "SGD":
    # code for training model using SGD
    optimizer = GD(lr = lr)
    losses, thetas, _ = train_mini_batch(X, Y, theta0,
                                  optimizer=optimizer,
                                  num_epochs=num_epochs,
                                  batch_size=5)
  else:
    # code for training model using Momentum
    optimizer = Momentum(lr = lr, moment=0.9)
    losses, thetas, _ = train_mini_batch(X, Y, theta0,
                                                 optimizer=optimizer,
                                                 num_epochs=num_epochs,
                                                 batch_size = 5)
  # Step 2: Plot the figures and return
  figLosses = go.Figure(plot_losses(losses))
  figGrads = go.Figure(plot_gradients(thetas, losses))

  return figLosses, figGrads

In [22]:
## Run the app internally
# PORT = 8866
# app.run(port=PORT, debug=True)

# Run the app externally
PORT = 8866
app.run(port=PORT, debug=False, jupyter_mode="external")

Dash app running on:


<IPython.core.display.Javascript object>